# 建立所有商品名稱 + 商品分類細項 的list

In [2]:
import pandas as pd
df1 = pd.read_csv('./dataset/products.csv')
df2 = pd.read_csv('./dataset/aisles.csv')
df3 = df1.merge(df2, on=['aisle_id'], how='left')
d = pd.Series(df3.aisle.values, index = df3.product_name).to_dict()
items=[]
for key, value in d.items():
    item = key + ' ' + value
    items.append(item)
items[0:5]

['Chocolate Sandwich Cookies cookies cakes',
 'All-Seasons Salt spices seasonings',
 'Robust Golden Unsweetened Oolong Tea tea',
 'Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce frozen meals',
 'Green Chile Anytime Sauce marinades meat preparation']

# 取得Product info Dict

In [155]:
def getInfo(search_result):
    for item_info in search_result:

        product_info = {
            "keyword": "",
            "name": "",
            "brand": "",
            "url": "",
            "pic": "",
            "price": "",
            "category": [],
            "star_ratings": [],
            "other_ratings": {},
            "at_a_glance": [],
            "highlights": [],
            "specifications": {},
            "description": "",
            "reviews": []
        }

        p_name = item_info.get('title', 'NA')
        p_brand = item_info.get('brand', 'NA')
        p_price = item_info.get('price').get('current_retail', 'NA')
        p_glances = item_info.get("wellness_merchandise_attributes", 'NA')
        p_description = item_info.get('description', 'NA')

        #rating
        p_ratings_s_avg = item_info.get('average_rating', 'NA')
        p_ratings_s_cnt = item_info.get('total_reviews', 'NA')

        p_url = lambda x: 'https://www.target.com'+x if item_info.get('url') else 'NA'
        p_pic = lambda x: x.get('base_url')+x.get('primary') if item_info.get('images') else 'NA'
        p_highlights = lambda x:x['soft_bullets'].get('bullets', 'NA') if item_info.get('soft_bullets') else 'NA' 

        #category
        req = requests.get(p_url(item_info['url']))
        soup = BeautifulSoup(req.text, "html")
        p_category = soup.find("div", {"class", "h-text-sm h-padding-v-tiny"}).text.strip("\u200e").split("/")

        #reviews
        p_review = item_info.get("top_reviews", 'NA')

        product_info['keyword'] = item
        product_info['name'] = p_name
        product_info['url'] = p_url(item_info['url'])
        product_info['brand'] = p_brand
        product_info['pic'] = p_pic(item_info['images'][0])
        product_info['price'] = p_price
        product_info['category'] = p_category[1:]
        product_info['star_ratings'].append(p_ratings_s_avg) 
        product_info['star_ratings'].append(p_ratings_s_cnt) 
        product_info['other_ratings'] 

        #specifications
        spec_list = ["Contains", "Form", "State of Readiness", "Store", "Package Quantity", "Package type", "Net weight"]
        if item_info.get("bullet_description"):
            for i in spec_list:
                for j in item_info["bullet_description"]:
                    j = re.sub("<.*?>", " ",j).strip(' ')
                    if re.match(i, j.split(": ")[0]):
                        product_info["specifications"][i] = j.split(": ")[1]
        else:
            product_info["specifications"]='NA'

        #description
        product_info['description'] = p_description
        product_info['description'] = re.sub("<.*?>", " ",product_info['description'])

        #reviews
        if (p_review == "NA"):
            product_info["reviews"] = "NA"
        else:
            for rev in p_review:
                review = re.sub('[\r\n\t]', '', rev["review_text"])
                product_info["reviews"].append(review)

        #other_ratings
        if item_info.get("secondary_ratings_averages"):
            for ratings in item_info["secondary_ratings_averages"].values():
                product_info["other_ratings"][ratings["Id"]] = ratings["AverageRating"]
        else:
            product_info["other_ratings"]='NA'

        #at_a_glance
        if (p_glances=='NA'):
            product_info['at_a_glance']= 'NA'
        else:
            for glance in p_glances:
                product_info['at_a_glance'].append(glance["value_name"])

        #highlight
        if (p_highlights(item_info)=='NA'):
            product_info['highlights'] = 'NA'
        else:
            for highlight in p_highlights(item_info):
                product_info['highlights'].append(highlight)

        products.append(product_info)



# 解析網址 + 更改參數 >> get info

In [166]:
import requests
import json
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse
import random

In [167]:
url = 'https://redsky.target.com/v2/plp/search/?channel=web&count=6&default_purchasability_filter=true&facet_recovery=false&fulfillment_test_mode=grocery_opu_team_member_test&isDLP=false&keyword=Chocolate+Sandwich+Cookies&offset=0&pageId=%2Fs%2FChocolate+Sandwich+Cookies&pricing_store_id=1357&store_ids=1357&visitorId=017312BF083B020182249D1E7E519F0E&include_sponsored_search_v2=true&ppatok=AOxT33a&platform=desktop&useragent=Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F83.0.4103.116+Safari%2F537.36&excludes=available_to_promise_qualitative%2Cavailable_to_promise_location_qualitative&key=eb2551e4accc14f38cc42d32fbc2b2ea'

item=''
products = []

#random testing
testing_items = random.sample(items, 5)

for item in testing_items:
    url = urlparse(url)
    list_url = list(url)
    #set new params
    params = dict( (k, v if len(v)>1 else v[0] ) for k, v in parse_qs(url.query).items() )
    params['include_sponsored_search_v2'] = 'false'
    params['keyword'] = item
    params['pageId'] = '/s/%s'%(item)

    #update url
    list_url[4] = urlencode(params)
    url = urlunparse(list_url)
    item=item
    print(item, url)
    
    useragent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
    headers = {'User-Agent':useragent}

    res = requests.get(url, headers=headers)
    json_str = res.text
    js = json.loads(json_str)
    js_list = list(js.values())

    search_result = js_list[1]['items']['Item']

    getInfo(search_result)
    print(products)


G Series Performance Cool Blue Sports Drink energy sports drinks https://redsky.target.com/v2/plp/search/?channel=web&count=6&default_purchasability_filter=true&facet_recovery=false&fulfillment_test_mode=grocery_opu_team_member_test&isDLP=false&keyword=G+Series+Performance+Cool+Blue+Sports+Drink+energy+sports+drinks&offset=0&pageId=%2Fs%2FG+Series+Performance+Cool+Blue+Sports+Drink+energy+sports+drinks&pricing_store_id=1357&store_ids=1357&visitorId=017312BF083B020182249D1E7E519F0E&include_sponsored_search_v2=false&ppatok=AOxT33a&platform=desktop&useragent=Mozilla%2F5.0+%28Windows+NT+10.0%3B+Win64%3B+x64%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F83.0.4103.116+Safari%2F537.36&excludes=available_to_promise_qualitative%2Cavailable_to_promise_location_qualitative&key=eb2551e4accc14f38cc42d32fbc2b2ea
[{'keyword': 'G Series Performance Cool Blue Sports Drink energy sports drinks', 'name': 'Gatorade Cool Blue Sports Drink - 32 fl oz Bottle', 'brand': 'Gatorade', 'url': 'https:

[{'keyword': 'G Series Performance Cool Blue Sports Drink energy sports drinks', 'name': 'Gatorade Cool Blue Sports Drink - 32 fl oz Bottle', 'brand': 'Gatorade', 'url': 'https://www.target.com/p/gatorade-cool-blue-sports-drink-32-fl-oz-bottle/-/A-13674092', 'pic': 'https://target.scene7.com/is/image/Target/GUEST_184c1215-1ce4-49d2-9338-3b3d39793b1c', 'price': 0.99, 'category': ['Grocery', 'Beverages', 'Energy & Sports Drinks'], 'star_ratings': [5.0, 6], 'other_ratings': {'Value': 5.0, 'Taste': 5.0, 'Quality': 5.0}, 'at_a_glance': 'NA', 'highlights': ['Includes 1 (32oz) bottles of Gatorade Thirst Quencher Sports Drink, Cool Blue flavor', "When you sweat, you lose more than water. Gatorade Thirst Quencher contains critical electrolytes to help replace what's lost in sweat.", "Top off your fuel stores with carbohydrate energy, your body's preferred source of fuel.", 'Formulated and tested at the Gatorade Sports Science Institute.', 'Gatorade is the official sports drink provider of the N

[{'keyword': 'G Series Performance Cool Blue Sports Drink energy sports drinks', 'name': 'Gatorade Cool Blue Sports Drink - 32 fl oz Bottle', 'brand': 'Gatorade', 'url': 'https://www.target.com/p/gatorade-cool-blue-sports-drink-32-fl-oz-bottle/-/A-13674092', 'pic': 'https://target.scene7.com/is/image/Target/GUEST_184c1215-1ce4-49d2-9338-3b3d39793b1c', 'price': 0.99, 'category': ['Grocery', 'Beverages', 'Energy & Sports Drinks'], 'star_ratings': [5.0, 6], 'other_ratings': {'Value': 5.0, 'Taste': 5.0, 'Quality': 5.0}, 'at_a_glance': 'NA', 'highlights': ['Includes 1 (32oz) bottles of Gatorade Thirst Quencher Sports Drink, Cool Blue flavor', "When you sweat, you lose more than water. Gatorade Thirst Quencher contains critical electrolytes to help replace what's lost in sweat.", "Top off your fuel stores with carbohydrate energy, your body's preferred source of fuel.", 'Formulated and tested at the Gatorade Sports Science Institute.', 'Gatorade is the official sports drink provider of the N

[{'keyword': 'G Series Performance Cool Blue Sports Drink energy sports drinks', 'name': 'Gatorade Cool Blue Sports Drink - 32 fl oz Bottle', 'brand': 'Gatorade', 'url': 'https://www.target.com/p/gatorade-cool-blue-sports-drink-32-fl-oz-bottle/-/A-13674092', 'pic': 'https://target.scene7.com/is/image/Target/GUEST_184c1215-1ce4-49d2-9338-3b3d39793b1c', 'price': 0.99, 'category': ['Grocery', 'Beverages', 'Energy & Sports Drinks'], 'star_ratings': [5.0, 6], 'other_ratings': {'Value': 5.0, 'Taste': 5.0, 'Quality': 5.0}, 'at_a_glance': 'NA', 'highlights': ['Includes 1 (32oz) bottles of Gatorade Thirst Quencher Sports Drink, Cool Blue flavor', "When you sweat, you lose more than water. Gatorade Thirst Quencher contains critical electrolytes to help replace what's lost in sweat.", "Top off your fuel stores with carbohydrate energy, your body's preferred source of fuel.", 'Formulated and tested at the Gatorade Sports Science Institute.', 'Gatorade is the official sports drink provider of the N

[{'keyword': 'G Series Performance Cool Blue Sports Drink energy sports drinks', 'name': 'Gatorade Cool Blue Sports Drink - 32 fl oz Bottle', 'brand': 'Gatorade', 'url': 'https://www.target.com/p/gatorade-cool-blue-sports-drink-32-fl-oz-bottle/-/A-13674092', 'pic': 'https://target.scene7.com/is/image/Target/GUEST_184c1215-1ce4-49d2-9338-3b3d39793b1c', 'price': 0.99, 'category': ['Grocery', 'Beverages', 'Energy & Sports Drinks'], 'star_ratings': [5.0, 6], 'other_ratings': {'Value': 5.0, 'Taste': 5.0, 'Quality': 5.0}, 'at_a_glance': 'NA', 'highlights': ['Includes 1 (32oz) bottles of Gatorade Thirst Quencher Sports Drink, Cool Blue flavor', "When you sweat, you lose more than water. Gatorade Thirst Quencher contains critical electrolytes to help replace what's lost in sweat.", "Top off your fuel stores with carbohydrate energy, your body's preferred source of fuel.", 'Formulated and tested at the Gatorade Sports Science Institute.', 'Gatorade is the official sports drink provider of the N